In [1]:
import tensorflow as tf
import os
import numpy as np
# No necesitaremos ImageDataGenerator para esta forma de cargar los datos
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import Xception, InceptionV3, ResNet50
from tensorflow.keras.layers import Input, concatenate, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 1.1. Verificar GPU (opcional, pero buena práctica en Kaggle Notebooks)
print("GPU available:", tf.config.list_physical_devices('GPU'))
if tf.config.list_physical_devices('GPU'):
    print("You're using a GPU! This will speed up training.")
else:
    print("No GPU found. Training will be slow.")

# 1.2. Definir las rutas base a tu dataset en Kaggle
# Basado en la imagen que proporcionaste
DATASET_ROOT_DIR = '/kaggle/input/food41/' # Ruta base donde está la carpeta food-101

# Rutas a las subcarpetas clave
IMAGES_DIR = os.path.join(DATASET_ROOT_DIR, 'images')
META_DIR = os.path.join(DATASET_ROOT_DIR, 'meta', 'meta') # Observa la doble carpeta 'meta'

# Rutas a los archivos de split
TRAIN_FILE = os.path.join(META_DIR, 'train.txt')
TEST_FILE = os.path.join(META_DIR, 'test.txt')
CLASSES_FILE = os.path.join(META_DIR, 'classes.txt')

BATCH_SIZE = 32

print(f"\nDataset root directory: {DATASET_ROOT_DIR}")
print(f"Images directory: {IMAGES_DIR}")
print(f"Meta directory: {META_DIR}")
print(f"Train file: {TRAIN_FILE}")
print(f"Test file: {TEST_FILE}")
print(f"Classes file: {CLASSES_FILE}")

2025-07-19 22:17:58.545319: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752963478.769859      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752963478.837536      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


GPU available: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
You're using a GPU! This will speed up training.

Dataset root directory: /kaggle/input/food41/
Images directory: /kaggle/input/food41/images
Meta directory: /kaggle/input/food41/meta/meta
Train file: /kaggle/input/food41/meta/meta/train.txt
Test file: /kaggle/input/food41/meta/meta/test.txt
Classes file: /kaggle/input/food41/meta/meta/classes.txt


In [2]:
# 2.1. Cargar nombres de clases y crear mapeo a etiquetas numéricas
class_names = []
with open(CLASSES_FILE, 'r') as f:
    for line in f:
        class_names.append(line.strip()) # Eliminar espacios en blanco y saltos de línea

class_to_idx = {name: i for i, name in enumerate(class_names)}
NUM_CLASSES = len(class_names)
print(f"\nDetected classes ({NUM_CLASSES}): {class_names}")

# 2.2. Función para leer los archivos train.txt/test.txt y obtener rutas y etiquetas
def get_image_paths_and_labels_from_txt(file_path, base_images_dir, class_to_idx):
    image_paths = []
    labels = []
    with open(file_path, 'r') as f:
        for line in f:
            relative_path = line.strip() # Ej: 'apple_pie/1000.jpg'
            
            # Extraer el nombre de la clase de la ruta relativa
            class_name = relative_path.split('/')[0]
            label = class_to_idx[class_name] # Mapear a entero

            # Construir la ruta completa a la imagen
            full_path = os.path.join(base_images_dir, relative_path + '.jpg') # Asume .jpg, Food-101 usa .jpg
            
            image_paths.append(full_path)
            labels.append(label)
    return image_paths, labels

# Obtener rutas y etiquetas para entrenamiento y prueba
print("\nLoading training image paths and labels...")
train_image_paths, train_labels = get_image_paths_and_labels_from_txt(TRAIN_FILE, IMAGES_DIR, class_to_idx)
print(f"Training images found: {len(train_image_paths)}")

print("Loading test image paths and labels...")
test_image_paths, test_labels = get_image_paths_and_labels_from_txt(TEST_FILE, IMAGES_DIR, class_to_idx)
print(f"Test images found: {len(test_image_paths)}")

# 2.3. Función de carga y preprocesamiento para tf.data
# Esta función leerá una imagen y la redimensionará a ambos tamaños requeridos.
def load_and_preprocess(image_path, label):
    # Cargar la imagen
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3) # Decodifica JPEG a 3 canales (RGB)
    
    # Redimensionar y normalizar la imagen para ambas entradas
    image_299 = tf.image.resize(image, (299, 299)) / 255.0
    image_224 = tf.image.resize(image, (224, 224)) / 255.0
    
    # Devuelve un diccionario para las múltiples entradas del modelo y la etiqueta
    return {'input_299x299': image_299, 'input_224x224': image_224}, label

# 2.4. Crear los objetos tf.data.Dataset
# Crea datasets a partir de las rutas y etiquetas
train_ds = tf.data.Dataset.from_tensor_slices((tf.constant(train_image_paths), tf.constant(train_labels)))
test_ds = tf.data.Dataset.from_tensor_slices((tf.constant(test_image_paths), tf.constant(test_labels)))

# Configurar el pipeline de datos para eficiencia
# `shuffle`: Mezcla los datos para el entrenamiento (solo en train_ds).
# `map`: Aplica la función de preprocesamiento a cada elemento. `num_parallel_calls` acelera esto.
# `batch`: Agrupa los elementos en batches.
# `prefetch`: Superpone el preprocesamiento de datos con el entrenamiento del modelo.
train_ds = train_ds.shuffle(buffer_size=len(train_image_paths)).map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(load_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print("\ntf.data.Dataset for training and testing are ready.")


Detected classes (101): ['apple_pie', 'baby_back_ribs', 'baklava', 'beef_carpaccio', 'beef_tartare', 'beet_salad', 'beignets', 'bibimbap', 'bread_pudding', 'breakfast_burrito', 'bruschetta', 'caesar_salad', 'cannoli', 'caprese_salad', 'carrot_cake', 'ceviche', 'cheesecake', 'cheese_plate', 'chicken_curry', 'chicken_quesadilla', 'chicken_wings', 'chocolate_cake', 'chocolate_mousse', 'churros', 'clam_chowder', 'club_sandwich', 'crab_cakes', 'creme_brulee', 'croque_madame', 'cup_cakes', 'deviled_eggs', 'donuts', 'dumplings', 'edamame', 'eggs_benedict', 'escargots', 'falafel', 'filet_mignon', 'fish_and_chips', 'foie_gras', 'french_fries', 'french_onion_soup', 'french_toast', 'fried_calamari', 'fried_rice', 'frozen_yogurt', 'garlic_bread', 'gnocchi', 'greek_salad', 'grilled_cheese_sandwich', 'grilled_salmon', 'guacamole', 'gyoza', 'hamburger', 'hot_and_sour_soup', 'hot_dog', 'huevos_rancheros', 'hummus', 'ice_cream', 'lasagna', 'lobster_bisque', 'lobster_roll_sandwich', 'macaroni_and_chees

I0000 00:00:1752963500.893891      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1752963500.894570      36 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


In [ ]:
# 3.1. Definir las capas de entrada para cada tamaño de imagen
input_299 = Input(shape=(299, 299, 3), name='input_299x299')
input_224 = Input(shape=(224, 224, 3), name='input_224x224')

# 3.2. Cargar modelos base pre-entrenados (con pesos de ImageNet) y congelar sus capas
# Usaremos `pooling='avg'` para añadir una capa GlobalAveragePooling2D y aplanar las características.

# Xception
print("\nConfiguring Xception base...")
base_xception = Xception(weights='imagenet', include_top=False, input_tensor=input_299, pooling='avg')
for layer in base_xception.layers:
    layer.trainable = False # Congelar las capas de la base pre-entrenada
xception_output = base_xception.output

# InceptionV3
print("Configuring InceptionV3 base...")
base_inception = InceptionV3(weights='imagenet', include_top=False, input_tensor=input_299, pooling='avg')
for layer in base_inception.layers:
    layer.trainable = False # Congelar las capas
inception_output = base_inception.output

# ResNet50
print("Configuring ResNet50 base...")
base_resnet = ResNet50(weights='imagenet', include_top=False, input_tensor=input_224, pooling='avg')
for layer in base_resnet.layers:
    layer.trainable = False # Congelar las capas
resnet_output = base_resnet.output

print("\nAll base models configured and frozen.")

# 3.3. Concatenar las características de los tres modelos
merged_features = concatenate([xception_output, inception_output, resnet_output])

# 3.4. Añadir una nueva cabeza de clasificación (capas densas)
x = Dense(512, activation='relu')(merged_features)
x = Dropout(0.5)(x) # Capa de dropout para regularización
output_layer = Dense(NUM_CLASSES, activation='softmax')(x) # Capa de salida con activación softmax

# 3.5. Crear el modelo híbrido final
hybrid_model = Model(inputs=[input_299, input_224], outputs=output_layer)

# 3.6. Compilar el modelo
hybrid_model.compile(optimizer='adam',
                     loss='sparse_categorical_crossentropy', # Usar para etiquetas de enteros
                     metrics=['accuracy'])

print("\nSummary del Modelo Híbrido (solo la nueva cabeza se entrenará):")
hybrid_model.summary()


Configuring Xception base...
83683744/83683744 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Configuring InceptionV3 base...
87910968/87910968 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
Configuring ResNet50 base...
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step

All base models configured and frozen.

Summary del Modelo Híbrido (solo la nueva cabeza se entrenará):


Model: "functional"

In [4]:
# 4.1. Crear directorio para guardar checkpoints en tu Kaggle /kaggle/working/
checkpoint_dir = '/kaggle/working/food101_hybrid_checkpoints'
os.makedirs(checkpoint_dir, exist_ok=True)
print(f"\nCheckpoint directory created at: {checkpoint_dir}")

# 4.2. Definir el callback ModelCheckpoint
checkpoint_filepath = os.path.join(checkpoint_dir, 'best_hybrid_food101_model.h5')
model_checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# 4.3. Definir el callback EarlyStopping
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    mode='max',
    restore_best_weights=True,
    verbose=1
)

# 4.4. Agrupar los callbacks en una lista
callbacks_list = [model_checkpoint_callback, early_stopping_callback]


Checkpoint directory created at: /kaggle/working/food101_hybrid_checkpoints


In [5]:
# 5.1. Definir el número máximo de épocas
EPOCHS = 15

# 5.2. Iniciar el entrenamiento
print("\nStarting the training of the Hybrid Model's classification head...")
history = hybrid_model.fit(
    train_ds,              # Ahora pasamos directamente el tf.data.Dataset
    epochs=EPOCHS,
    validation_data=test_ds, # Y aquí el tf.data.Dataset de validación/prueba
    callbacks=callbacks_list
)

print("\nHybrid Model training completed.")


Starting the training of the Hybrid Model's classification head...
Epoch 1/15


I0000 00:00:1752963563.817687      97 service.cc:148] XLA service 0x7e006c0da4b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752963563.818501      97 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1752963563.818520      97 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1752963567.891408      97 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-07-19 22:19:33.855202: E external/local_xla/xla/service/slow_operation_alarm.cc:65] Trying algorithm eng3{k11=0} for conv (f32[32,128,147,147]{3,2,1,0}, u8[0]{0}) custom-call(f32[32,128,147,147]{3,2,1,0}, f32[128,128,1,1]{3,2,1,0}), window={size=1x1}, dim_labels=bf01_oi01->bf01, custom_call_target="__cudnn$convForward", backend_config={"cudnn_conv_backend_config":{"activation_mode":"kNone","conv_result_scale":1,"leakyrelu_alpha":0,"side_input_scale":0},"force_earliest_schedule":false,"operation_queue_id":"0

2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 503ms/step - accuracy: 0.2018 - loss: 3.3954
Epoch 1: val_accuracy improved from -inf to 0.52123, saving model to /kaggle/working/food101_hybrid_checkpoints/best_hybrid_food101_model.h5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1671s 681ms/step - accuracy: 0.2019 - loss: 3.3952 - val_accuracy: 0.5212 - val_loss: 1.8932
Epoch 2/15
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.3607 - loss: 2.5169
Epoch 2: val_accuracy improved from 0.52123 to 0.56012, saving model to /kaggle/working/food101_hybrid_checkpoints/best_hybrid_food101_model.h5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1581s 668ms/step - accuracy: 0.3607 - loss: 2.5168 - val_accuracy: 0.5601 - val_loss: 1.7708
Epoch 3/15
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.3850 - loss: 2.3834
Epoch 3: val_accuracy improved from 0.56012 to 0.56851, saving model to /kaggle/working/food101_hybrid_checkpoints/best_hybrid_food101_model.h5
2368/2368 ━━━━━━━━━━━━━━━━━━━━ 1578s 666ms/step - accuracy: 0.38

In [7]:
# --- 6. Guardar el Modelo Final ---

# Define la ruta para guardar el modelo final
# Lo guardaremos en el mismo directorio persistente de Kaggle
final_model_save_path = '/kaggle/working/final_hybrid_food101_model.h5'

# Guarda el modelo. Dado que EarlyStopping.restore_best_weights=True,
# el `hybrid_model` ya contiene los pesos del mejor checkpoint.
print(f"\nSaving the final trained hybrid model to: {final_model_save_path}")
hybrid_model.save(final_model_save_path)
print("Final model saved successfully!")

# Puedes verificar que el archivo existe
if os.path.exists(final_model_save_path):
    print(f"Model file confirmed at: {final_model_save_path}")


Saving the final trained hybrid model to: /kaggle/working/final_hybrid_food101_model.h5
Final model saved successfully!
Model file confirmed at: /kaggle/working/final_hybrid_food101_model.h5
